In [3]:
# 加载HuggingFaceH4/MATH-500
from datasets import load_dataset
import json

# 加载数据集
dataset = load_dataset("HuggingFaceH4/MATH-500", split="test")

In [4]:
# 转换数据结构，保持 instruction 为空
llama_factory_data = []
for item in dataset:
    llama_factory_data.append({
        "instruction": r"Please solve the following math problem. Write your reasoning clearly using LaTeX. Box the final answer using \boxed{}.",
        "input": item["problem"].strip() if item["problem"] else "",
        "output": item["solution"].strip() if item["solution"] else ""
    })

# 保存为json文件，格式美观且utf-8编码
with open("math500.json", "w", encoding="utf-8") as f:
    json.dump(llama_factory_data, f, ensure_ascii=False, indent=2)

print("导出完成，文件名：math500.json")

导出完成，文件名：math500.json


In [13]:
# %% [markdown]
# ## Math500 → llama-factory 数据转换器
#
# **功能**
# 1. 读取原始题库（含题目与参考答案）
# 2. 读取模型预测结果 `generated_predictions-detailed-results.jsonl`
# 3. 判断每道题是否回答正确，生成 “Yes / No” 标签
# 4. 按 llama-factory 所需格式导出 `math500_yesno.json`
#
# **使用方法**
# - 把原始题库放到 `math500_original.json`（或自行修改路径）
# - 把预测结果放到 `generated_predictions-detailed-results.jsonl`
# - 运行所有单元，最终文件会保存在同级目录

# %%
import json
from pathlib import Path

# ----------------- 手动配置区 -----------------
DATASET_PATH   = Path("math500.json")          # 原始题库
PRED_PATH      = Path("generated_predictions-detailed-results.jsonl")  # 预测结果
OUTPUT_PATH    = Path("math500_yesno.json")             # 导出文件
INSTRUCTION_TXT = (
    "Please decide whether you know answer to the following question."
    "No need to provide any reasoning or explanation, just give the answer with one word 'Yes' or 'No'."
    "Please answer carefully and don't be too confident. If the question is difficult, we will find a stronger model to do it. It doesn't matter if you can't."
)
# ---------------------------------------------

# %%
def load_dataset(path: Path):
    """
    读取原始题库，返回 idx -> 题目信息 的 dict
    如果你的数据是 dict 形式（带 id），请自行修改返回键
    """
    with path.open(encoding="utf-8") as f:
        data = json.load(f)
    return {idx: item for idx, item in enumerate(data)}

# %%
def load_predictions(path: Path):
    """
    读取 jsonl 预测结果，返回 idx -> 是否答对 的 dict
    常见字段: accuracy / pass / is_correct
    如有其他字段，请在 correct 判断中补充
    """
    idx2correct = {}
    with path.open(encoding="utf-8") as f:
        for line in f:
            rec = json.loads(line)
            idx = rec.get("index")  # 根据实际字段决定使用 "index" 还是 "id"
            if idx is None:
                raise KeyError("无法在预测文件中找到 index/id 字段！")

            correct = (
                rec.get("accuracy", 0) > 0
                or rec.get("pass", False)
                or rec.get("is_correct", False)
            )
            idx2correct[idx] = correct
    return idx2correct

# %%
# 载入数据
problems      = load_dataset(DATASET_PATH)
pred_results  = load_predictions(PRED_PATH)

print(f"题目数量: {len(problems)}")
print(f"预测记录: {len(pred_results)}")

# %%
# 组装为 llama-factory 格式
llama_factory_data = []
for idx, item in problems.items():
    problem_text = (item.get("input") or item.get("problem") or "").strip()
    llama_factory_data.append({
        "instruction": INSTRUCTION_TXT,
        "input":  problem_text,
        "output": "Yes." if pred_results.get(idx, False) else "No."
    })

# %%
# 保存 JSON
with OUTPUT_PATH.open("w", encoding="utf-8") as f:
    json.dump(llama_factory_data, f, ensure_ascii=False, indent=2)

print(f"✅ 导出完成，文件名: {OUTPUT_PATH}")

# %%
# （可选）快速预览前 3 条
for sample in llama_factory_data[:3]:
    print(sample, "\n")


题目数量: 500
预测记录: 500
✅ 导出完成，文件名: math500_yesno.json
{'instruction': "Please decide whether you know answer to the following question.No need to provide any reasoning or explanation, just give the answer with one word 'Yes' or 'No'.Please answer carefully and don't be too confident. If the question is difficult, we will find a stronger model to do it. It doesn't matter if you can't.", 'input': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$', 'output': 'Yes.'} 

{'instruction': "Please decide whether you know answer to the following question.No need to provide any reasoning or explanation, just give the answer with one word 'Yes' or 'No'.Please answer carefully and don't be too confident. If the question is difficult, we will find a stronger model to do it. It doesn't matter if you can't.", 'input': 'Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} 